In [3]:
import sqlite3
import csv
import pandas as pd
import os
import numpy as np
import json

In [4]:
with open('hst_bibcode_prog_associations.json', 'r') as f:
    assoc = json.load(f)

In [200]:
with open('hst_progs.json', 'r') as f:
    progs = json.load(f)

In [586]:
print(len(assoc),type(assoc))

1 <class 'dict'>


In [7]:
assoc.keys()

dict_keys(['bibcodes'])

In [8]:
type(assoc['bibcodes'])

list

In [9]:
len(assoc['bibcodes'])

16498

In [10]:
df_assoc = pd.DataFrame(assoc['bibcodes'])
df_assoc.head(5)

,bibcode,flag,programs
0,1991AJ....102..654H,GO,[2887]
1,1991AJ....102.1553K,GO,[3121]
2,1991ApJ...369L..21B,AR,"[3025, 3029, 2979]"
3,1991ApJ...369L..27E,GO,[3001]
4,1991ApJ...369L..31L,AR,[3001]


In [242]:
type(progs['programs'])

list

In [243]:
progs['programs'][:3]

[{'program': 13450,
  'cycle': 21,
  'title': 'Separating the Spectral Components of the Massive Triple Star System Delta Orionis',
  'fname': 'Douglas',
  'mi': 'R.',
  'lname': 'Gies',
  'institution': 'Georgia State University Research Foundation                ',
  'abstract': 'The multiple star system of delta Orionis represents one of the closest examples of a luminous O-star with a strong stellar wind, and it was the target of a recent multi-wavelength campaign to determine the source of the wind X-ray emission.  It consists of aclose eclipsing binary with a more distant tertiary, and all the components are massive stars.  Investigations of the radial velocity curves of the eclipsing system are made difficult by severe line blending with the spectral lines of the tertiary star, and the resulting mass estimates range by a factor of two.  We propose that the solution to this problem is to isolate the flux of the tertiary through high angular resolutionspectroscopy with HST/STIS, a

In [204]:
df_progs = pd.DataFrame(progs['programs'])

In [296]:
6402 in df_progs['program']

True

In [11]:
typeof2 = [];
for i in range(len(df_assoc)):
    typeof2.append(type(df_assoc.loc[0, 'programs']))

In [207]:
df_progs.columns

Index(['abstract', 'cycle', 'fname', 'institution', 'lname', 'mi', 'program',
       'sci_cat', 'title'],
      dtype='object')

In [276]:
df_progs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11198 entries, 0 to 11197
Data columns (total 9 columns):
abstract       11198 non-null object
cycle          11198 non-null int64
fname          11198 non-null object
institution    11198 non-null object
lname          11198 non-null object
mi             11198 non-null object
program        11198 non-null int64
sci_cat        11193 non-null object
title          11198 non-null object
dtypes: int64(2), object(7)
memory usage: 787.4+ KB


In [12]:
len(np.unique(df_assoc['bibcode']))

16498

In [211]:
df_progs_trim = df_progs[['program','title']]

In [312]:
df_progs_trim.head(10)

,program,title
0,13450,Separating the Spectral Components of the Mass...
1,12288,Hot Evolved Companions to Intermediate-Mass Ma...
2,9840,Wind Accretion and State Transitions in the Bl...
3,8392,The Masses of the O-type Binary 15 Monocerotis...
4,8730,The Masses of the O-type Binary 15 Monocerotis
5,6267,PARALLAX OF FEIGE 24
6,15457,COS/FUV/G160M/1533 Wavelength Calibration - Di...
7,15485,CAL/STIS: Standard Stars for the JWST Mission ...
8,15530,ACS SBC Recovery from Anomalous Shutdown
9,15624,Gyro/FHST Alignment


In [594]:
len(df_progs_trim)

11198

In [313]:
df_progs_trim.loc[df_progs['program']==13450]

,program,title
0,13450,Separating the Spectral Components of the Mass...


In [288]:
dict_prog = {k: v for k, v in zip(df_progs['program'], df_progs['title'])}

In [305]:
df_progs.loc[lambda df_progs: df_progs['program']==6402]

,abstract,cycle,fname,institution,lname,mi,program,sci_cat,title


In [13]:
len(df_assoc['bibcode'])

16498

In [593]:
df_assoc['flag'].describe()

count     16346
unique        3
top          AR
freq       7011
Name: flag, dtype: object

In [14]:
df_assoc.loc[3,'programs']+df_assoc.loc[4,'programs']

['3001', '3001']

In [608]:
key_v1 = pd.read_csv('df_key_0629.csv')

In [609]:
key_v1 = key_v1[['bibcode','key_0629']]

In [58]:
conn = sqlite3.connect('hubble.sqlite')
cur = conn.cursor()

In [620]:
cur.execute('''DROP TABLE IF EXISTS publications''') 
cur.execute('''
            CREATE TABLE publications(
                id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                bibcode TEXT,
                keywords TEXT,
                FOREIGN KEY(bibcode) REFERENCES datasets(bibcode))
                ''')
conn.commit()

In [214]:
cur.execute('''DROP TABLE IF EXISTS progs''') 
cur.execute('''
            CREATE TABLE progs(
                programs TEXT,
                titles TEXT,
                PRIMARY KEY(programs))
                ''')
conn.commit()

In [60]:
cur.execute('''DROP TABLE IF EXISTS datasets''') 
cur.execute('''
           CREATE TABLE datasets(
                bibcode TEXT,
                programs TEXT,
                PRIMARY KEY(bibcode))
                ''')

conn.commit()

In [621]:
for index, row in key_v1.iterrows():
    try:
        cur.execute('''
                INSERT INTO publications(
                bibcode,
                keywords
                ) VALUES (?,?)''',(row[0],row[1])
               )
    except sqlite3.Error as e:
        print(e)
conn.commit()

In [62]:
for index, row in df_assoc.iterrows():
    try:
        cur.execute('''
                INSERT INTO datasets(
                bibcode,
                programs
                ) VALUES (?,?)''',(row[0], str(row[2]))
               )
    except sqlite3.Error as e:
        print (e)
conn.commit()

In [220]:
for index, row in df_progs_trim.iterrows():
    try:
        cur.execute('''
                INSERT INTO progs(
                programs,
                titles
                ) VALUES (?,?)''',(row[0], row[1])
               )
    except sqlite3.Error as e:
        print (e)
conn.commit()

In [622]:
cur.execute('''DROP TABLE IF EXISTS dataset_key''') 
cur.execute('''CREATE TABLE dataset_key AS
                SELECT publications.bibcode, publications.keywords, datasets.programs
                FROM publications JOIN datasets ON publications.bibcode = datasets.bibcode 
                ORDER BY publications.bibcode
            ''')

conn.commit()

In [616]:
trial = pd.read_sql_query("SELECT * FROM publications", conn)
trial.head()

,id,bibcode,title_key
0,1,1993ApJ...417..338H,['star individual constellation name ophiuchi'...
1,2,1993ApJ...404..124S,"['absorption spectra', 'astronomical spectrosc..."
2,3,1991ApJ...373L..55M,"['radio source galaxy', 'radio galaxy', 'astro..."
3,4,1993ApJ...409L...5B,"['cosmic dust', 'astronomy infrared', 'astroch..."
4,5,1991AJ....102..654H,"['space vehicle', 'astrophysic', 'galaxy gener..."


In [219]:
df_progs_trim.iloc[0,0]

13450

In [65]:
trial2 = pd.read_sql_query("SELECT * FROM datasets", conn)
len(trial2)

16498

In [232]:
trial3 = pd.read_sql_query("SELECT * FROM progs", conn)
len(trial3)

11198

In [623]:
df_join = pd.read_sql_query("SELECT * FROM dataset_key", conn)
len(df_join)

16405

In [624]:
df_join[:10]

,bibcode,keywords,programs
0,1991AJ....102..654H,"['space vehicle', 'astrophysic', 'galaxy gener...",['2887']
1,1991AJ....102.1553K,"['space vehicle', 'red shift', 'signal to nois...",['3121']
2,1991ApJ...369L..21B,"['spaceborne photography', 'instrumentation de...","['3025', '3029', '2979']"
3,1991ApJ...369L..27E,"['bar galaxy', 'galaxy bar', 'oxygen spectra',...",['3001']
4,1991ApJ...369L..31L,"['astrochemistry', 'oxygen spectra', 'astrophy...",['3001']
5,1991ApJ...369L..35H,"['space vehicle', 'star variable', 'technique ...","['3008', '3010', '3053', '3013', '2886']"
6,1991ApJ...369L..41L,"['luminosity', 'spiral galaxy', 'instrumentati...",['3009']
7,1991ApJ...369L..45L,"['main sequence star', 'astrophysic', 'cluster...",['3011']
8,1991ApJ...369L..51W,"['instrumentation detector', 'surface science'...",['2890']
9,1991ApJ...369L..55M,"['technique interferometric', 'instrumentation...",['2993']


In [625]:
import ast
program_id = [];
for i in range(len(df_join)):
    program_id = program_id + ast.literal_eval(df_join['programs'][i])
# converting the programs column to list of program IDs, and concatenate all the lists 
len(program_id)

41930

In [626]:
id_set = set(program_id)
len(id_set) ### the total number of unique unique IDs

6441

In [627]:
# convert string to list for 'programs' column
pro_list = [];
for _ in range(len(df_join)):
    pro_list.append(ast.literal_eval(df_join['programs'][_]))   
len(pro_list)

16405

In [628]:
# convert string to list for 'keywords' column
key_list = [];
for _ in range(len(df_join)):
    key_list.append(ast.literal_eval(df_join['keywords'][_]))   
len(key_list)

16405

In [633]:
# loop through the list of program list, in order to aggregate keywords from publications to 
# datasets, and store the citation of datasets in a different dictionary
id_key_dict = {}
id_count_dict = {}
for pro_id in id_set:
    id_key_dict[pro_id] = [];
    id_count_dict[pro_id] = 0
    for i in range(len(pro_list)):
        if pro_id in pro_list[i]:
            id_key_dict[pro_id] = id_key_dict[pro_id] + key_list[i];
            id_count_dict[pro_id] = id_count_dict[pro_id] + 1


In [632]:
len(pro_list)

16405

In [601]:
id_list = list(id_set)
%store id_list

Stored 'id_list' (list)


In [651]:
keyy = np.unique(id_key_dict['2887'], return_counts = True)[0].tolist()
repp = np.unique(id_key_dict['2887'], return_counts = True)[1].tolist()
dict_trial = {k: v for k, v in zip(keyy, repp)}

In [637]:
for iddd in dict_trial.keys():
    dict_trial[iddd] = (dict_trial[iddd]/id_count_dict['2887'], id_count_dict['2887'])
# dict_trial.value is tuple type, whose first element is frequency, second element is absolute count

In [605]:
id_list.index('9122')

1893

In [657]:
%store -r cal_list

In [658]:
#final_list is a list of dictionary, each dictionary is for one dataset, the dictionary is keywords: (freq, count)
final_list_1 = [];
for _id in id_key_dict:
    keyy = np.unique(id_key_dict[_id], return_counts = True)[0].tolist()
    repp = np.unique(id_key_dict[_id], return_counts = True)[1].tolist()
    dict_trial = {k: v for k, v in zip(keyy, repp)}
    mean_cit = np.mean(repp)
    for iddd in dict_trial.keys():
        citation = dict_trial[iddd]
        if iddd in cal_list:
            dict_trial[iddd] = (0.001*dict_trial[iddd]/id_count_dict[_id], id_count_dict[_id])
        elif citation > mean_cit:
            dict_trial[iddd] = ((citation-np.min(repp))/(np.max(repp)-np.min(repp))*dict_trial[iddd]/id_count_dict[_id], id_count_dict[_id])
        else:
            dict_trial[iddd] = (dict_trial[iddd]/id_count_dict[_id], id_count_dict[_id])
    final_list_1.append(dict_trial)    

In [660]:
len(final_list_1)

6441

In [661]:
v1_df = pd.DataFrame(final_list_1)
# Takes a couple minutes to run

In [662]:
v1_df.shape

(6441, 37335)

In [663]:
#all keywords
listofkey = v1_df.columns.tolist()

In [672]:
v1_df.shape

(6441, 37335)

In [686]:
#Examine the keywords that are too popular to be actual keywords (if a word appears in 80 percent of the datasets)
inte_columns = []
for i in range(len(listofkey)):
    if v1_df[listofkey[i]].isna().sum() <1288:
        inte_columns.append(listofkey[i])
        print(listofkey[i], v1_df[listofkey[i]].isna().sum())
len(inte_columns)

000 k 560
old metalpoor 1249
old metalrich 1110
old nova 1110
old open 1208
optical ir 1078
optical island 736
optical knot 625
optical light 474
optical line 341
optical longslit 248
optical luminosity 184
optical midinfrared 142
optical monitor 81
optical morphology 63
optical nearinfrar 48
optical nearinfrared 35
optical nearir 29
optical nir 21
optical nucleus 20
optical photometry 14
optical polarization 14
optical radio 6
optical source 6
optical spectra 5
optical spectroscopic 4
optical spectroscopy 2
optical spectrum 4
optical synchrotron 2
optical thickness 4
optical transient 5
optical ultraviolet 7
optical uv 10
optical variability 13
optical wavelength 8
optical xray 8
opticalinfrar 9
opticalir 8
optically 8
optically bright 8
optically faint 10
optically identify 15
optically luminous 15
optically select 23
optically thick 30
optically thin 23
optically visible 24
opticalnearinfrared 24
opticalnearir 23
opticalnir 21
opticaluv 23
optimal 28
optimization 28
optimize 24
orbi

76

In [687]:
#Examine the keywords that are rare
inte_columns2 = []
for i in range(len(listofkey)):
    if v1_df[listofkey[i]].isna().sum() < 6421:
        inte_columns2.append(listofkey[i])
        print(listofkey[i], v1_df.shape[0]-v1_df[listofkey[i]].isna().sum())
len(inte_columns2)

0075 7
01 7
010 mag 11
012 lt 11
0124 kev 12
015 mag 13
03 055 9
03 kev 10
0310 kev 14
038 kev 15
05 solar 15
055 07 7
06 lt 15
06 z 8
070 12
08 lt 9
10 co 13
10 source 13
10 twoimage 7
100 earlytype 11
100000 11
101 14
104 gran 11
1052 8
10sup7sup 8
110 solar 15
12 c 15
12 day 15
12 iip 10
12 pc 11
1200 7
1275 7
13 l 14
13 year 12
1317217 10
1369 7
137 lt 15
1370132 10
139 lt 7
13988 8
1399 9
14 lowredshift 8
15 pc 13
1500 15
159 9
16 cm 15
16 nearinfrar 10
160 km 15
1600 10
161 7
163296 8
165 7
17 lt 9
174708 7
17600 galaxy 8
18 galaxy 15
1851 7
1885 m31 8
189733b 9
18minute 7
19 cm 10
1961v 7
19901997 7
1991 7
1992 12
1994 13
1995 10
1996 8
19961997 9
19962004 8
19982011 8
1998bw 12
1999 9
1type dichotomy 8
200 000 10
2000 10
20000 k 14
20022006 9
2004 8
2004et 7
2006my 2006ov 7
2009 8
2010 12
2011 8
2012 14
2012 ultra 12
2014 13
2014j wide 7
2015 8
210 7
21st century 15
2218 8
23 15
23 23 7
23 arcmin 7
2366 7
24 cm 7
248 7
25 candel 11
250 8
253 7
259 7
264 9
27 9
273 9
2744 7
28 e

crater 13
critical density 14
crl 2688 10
crowded stellar 7
crowding 13
crucial information 13
crv debris 7
csub2sub 14
ctio 15
curious 10
current saturn 7
currently undergo 15
cv 9
damped lyman 13
dark object 10
dawn 10
day burst 10
dead 8
death massive 11
debris disc 12
decade radius 8
decipher 8
decline relationship 7
decline star 13
decoupling 13
deduced 15
deep acs 7
deep colourmagnitude 9
deep groth 14
deep hstac 12
deep irac 12
deep kds 9
deep north 9
deep search 15
deep south 13
deep2 9
deflect 14
deimos 12
deja 7
del 12
delaytime distribution 11
demonstrate importance 14
dense matter 9
density contrast 13
density perturbation 9
dependence environmental 8
deplete core 7
depletion timescale 13
deredden 10
description reduction 14
desorption 15
detailed photometric 8
detailed structure 14
detectable signature 9
detection companion 7
detection highredshift 8
detection lyman 7
detection multiple 9
detection photometry 8
detection weak 14
deuterium interstellar cloud 11
deuterium so

galaxy individual haro 11 12
galaxy individual holmberg ii 10
galaxy individual i zw 18 13
galaxy individual ic 1613 7
galaxy individual ic 342 7
galaxy individual m 32 7
galaxy individual m 33 ngc 3338 ngc 3810 ngc 5457 ngc 6503 ngc 6946 10
galaxy individual m 51 7
galaxy individual m 81 15
galaxy individual m31 m33 12
galaxy individual m32 7
galaxy individual m51 12
galaxy individual m83 10
galaxy individual mac j114952223 7
galaxy individual markarian 3 8
galaxy individual messy number m32 11
galaxy individual messy number m82 12
galaxy individual mrk 231 12
galaxy individual mrk 259 7
galaxy individual mrk 573 8
galaxy individual name markarian 509 9
galaxy individual name markarian 78 7
galaxy individual name small magellanic cloud 9
galaxy individual ngc 1023 8
galaxy individual ngc 1052 11
galaxy individual ngc 1068 13
galaxy individual ngc 1097 8
galaxy individual ngc 1275 7
galaxy individual ngc 1316 7
galaxy individual ngc 1399 9
galaxy individual ngc 1569 13
galaxy individua

infrared view 11
infraredluminous merger 7
initial helium 14
inner arcsec 13
inner bulge 7
inner circumstellar 10
inner coma 7
inner mass 15
inner polar 7
inner radius 14
insight complex 8
instrument contribution 13
instrumental systematic 10
instrumentation antenna 12
instrumentation camera 15
instrumentation photometer 7
integrate line 10
integrate photometry 10
integrated spectra 12
intensity fluctuation 7
interact merge 8
interact pair 15
interact wind 10
interamerican optical 9
interferometer network 14
interferometer pdbi 15
interferometric astrometry 9
intergalactic matter 8
intergalactic medium cosmology theory largescale structure of universe 9
intermediate stage 13
internal dynamic 10
internally 11
interrelation 9
interstellar dust grain 8
interstellar krypton 11
interstellar oxygen 9
intracluster 15
io atmosphere 13
io atmospheric 14
ionizing luminosity 12
ionosphere 11
ionosphere planetary ionosphere 5435 8
ios 9
ios atmosphere 7
ir image 15
iracdetect 9
irregular dwarf 13


nearir search 8
nearultraviolet spectra 13
nearultraviolet spectroscopy 7
nebula m43 11
nebulae coolingcore 7
nebulae geometrically 8
negative feedback 14
neon 11
neptune rotational 11
neural network 8
neutral phosphoru 11
neutrino 7
nextgeneration 15
ngc 1023 11
ngc 121 8
ngc 1316 13
ngc 1569 14
ngc 1614 7
ngc 1705 14
ngc 205 13
ngc 2110 15
ngc 288 7
ngc 3067 8
ngc 3242 7
ngc 346 11
ngc 362 7
ngc 3690 11
ngc 3998 10
ngc 40384039 9
ngc 4244 7
ngc 4261 14
ngc 4278 10
ngc 4365 15
ngc 4406 15
ngc 4494 9
ngc 4559 9
ngc 4565 7
ngc 4579 7
ngc 4649 12
ngc 4696 7
ngc 5102 8
ngc 5252 9
ngc 5474 11
ngc 5477 10
ngc 55 7
ngc 5584 7
ngc 6090 8
ngc 6240 12
ngc 6251 8
ngc 6362 8
ngc 6541 7
ngc 6543 9
ngc 6624 12
ngc 6656 12
ngc 6720 9
ngc 6822 9
ngc 6951 9
ngc 7027 7
ngc 7252 14
ngc 7469 13
ngc300 7
ngc6528 10
ngc6553 8
nicmo archive 8
nicmo deep 9
nicmo grism 10
nicmo hdf 7
nicmos search 15
nightside magnetosphere 7
nir imaging 14
nitrogen chemistry 7
nitrogen oxygen 7
nlte model 13
nonaxisymmetric 

role environment 9
rosat pspc 11
rosetta 7
rossi xray 10
rotation pattern 11
rotation random 7
rotational modulation 13
roughly factor 13
rover 9
run vacuum 9
russell and colour 11
rw aur 15
rxte 15
s0 9
sagittarius 7
sagittarius dwarf 12
sao 206462 7
satellite observation 8
satellite pluto 7
satellite surface 11
satelliteborne instrument 9
saturn atmosphere 14
saturn auroral 7
saturn b 8
saturn equatorial 8
saturn magnetosphere 10
saturn satellite 9
saturn southern 12
sbs 0335052 12
scale hundred 14
scale key 9
scale law 15
scale otype 10
scaling relationship 8
science mission 13
scuba galaxy 8
sdss gem 8
sdsscosmo 12
search companion 8
search fail 11
search galaxygalaxy 10
search intermediatemass 7
search supernovae 7
seasonal change 13
secondary eclipse 9
secondgeneration star 11
secular change 8
selection highredshift 15
selection highz 8
selenium 7
selfenrichment 13
selfsimilar model 14
semiamplitude 13
sensor astrometry 10
sensor search 10
separate component 14
separation lt 10
s

testing model 9
texas 7
the galaxy solar neighbourhood 11
thin and thick disk 8
thin disc 15
thin shell 13
thing 14
threedimensional model 8
threedimensional smoothed 10
thus important 14
tidal radii 9
tilt effect 8
time explorer 13
time resolution 15
timeaverag 10
timedelay 10
timeresolv ultraviolet 8
timeserie photometry 15
titan haze 8
tomography 7
toomre sequence 8
tpagb star 15
trace inner 14
transit light 12
transitional disk 9
translucent cloud 13
translucent sight 9
transport mix 7
trapezium 13
trapezium cluster 11
treasury phat 9
treatment mix 15
tripletbas 9
triton 15
true opposition 8
tu 7
turnoff emsto 13
turnoff msto 7
tw hydrae 8
tycho2 14
tycho2 photometry 14
type ia supernova 8
typeia supernovae 7
u gem 10
ugc 9405 10
ulig 12
ulirgs 9
ultracompact xray 9
ultracool binary 8
ultradeep k 9
ultradeep nearinfrar 7
ultrafaint 9
ultraviolet aurora 9
ultraviolet emissionline 14
ultraviolet extinction 11
ultraviolet photometry 11
ultraviolet variability 7
ultravioletbright 11
ul

2708

In [819]:
# this block is to shrink dataframe size, so it can run on unix miro on aws
# if the website server has more memory, no need to run this cell
kept_columns = []
for i in range(len(listofkey)):
    if v1_df[listofkey[i]].isna().sum() < 6380 and v1_df[listofkey[i]].isna().sum() >3220:
        kept_columns.append(listofkey[i])
len(kept_columns)

3788

In [820]:
'supernovae' in kept_columns

True

In [821]:
'gravitational lensing' in kept_columns

True

In [822]:
'galaxy evolution' in kept_columns

True

In [823]:
v2_df = v1_df[kept_columns]
v2_df['ds_id'] = list(id_key_dict.keys())

/Users/yanfeiwang/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
prog_titles = [];
for _ in list(id_key_dict.keys()):
    try:
        prog_titles.append(dict_prog[int(_)])
    except:
        prog_titles.append('title not available')
len(prog_titles)

In [824]:
v2_df['title'] = prog_titles

/Users/yanfeiwang/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [711]:
cal_index = v2_df.index[v2_df.ds_id.isin(cal_list)].tolist()

In [825]:
# this block is to shrink dataframe size, so it can run on unix miro on aws
v2_df = v2_df.drop(cal_index, axis = 0)

In [826]:
v2_df.shape

(6126, 3790)

In [827]:
v2_df.to_csv('/Users/yanfeiwang/Downloads/df_v2.csv', index=False)